#part 1

import libraries

In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
!pip install geocoder
import geocoder
!pip install geopy
from geopy.geocoders import Nominatim 
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

scarpe wikioedia page

In [12]:
# send HTTP request to Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url).text

# parse the page, locate HTML tags with relevant information
soup = BeautifulSoup(response, 'lxml')
table = soup.find('table')
fields = table.find_all('td')

In [13]:
# empty lists to store data
postalcode = []
borough = []
neighborhood = []

# loop through all <td> tags to extract data from the table
for td in range(0, len(fields), 3):
    postalcode.append(fields[td].text.strip())
    borough.append(fields[td+1].text.strip())
    neighborhood.append(fields[td+2].text.strip())

# create a dataframe
df = pd.DataFrame({
    'PostalCode':postalcode,
    'Borough':borough,
    'Neighborhood':neighborhood
})
print(f"Dataframe contains {df.shape[0]} rows and {df.shape[1]} columns.")
df.head()

Dataframe contains 60 rows and 3 columns.


,PostalCode,Borough,Neighborhood
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods)
1,M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights)
2,M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
3,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North
4,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn)


rearrange info in table

In [14]:
df['Neighborhood'] = df['Neighborhood'].map(lambda x: str(x)[3:])
df['Borough'] = df['Borough'].map(lambda x: str(x)[3:])
df['PostalCode'] =df['PostalCode'].apply(lambda x:x[:3])
df 

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,North York(Parkwoods)
1,M4A,Downtown Toronto(Regent Park / Harbourfront),North York(Lawrence Manor / Lawrence Heights)
2,M7A,Not assigned,Etobicoke(Islington Avenue)
3,M1B,Not assigned,North York(Don Mills)North
4,M4B,"Downtown Toronto(Garden District, Ryerson)",North York(Glencairn)
5,M7B,Not assigned,Etobicoke(West Deane Park / Princess Gardens /...
6,M1C,Not assigned,North York(Don Mills)South(Flemingdon Park)
7,M4C,Downtown Toronto(St. James Town),York(Humewood-Cedarvale)
8,M7C,Not assigned,Etobicoke(Eringate / Bloordale Gardens / Old B...
9,M1E,Not assigned,Not assigned


remove boroughs that are 'Not assigned'

In [15]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
print(f"Dataframe now contains {df.shape[0]} rows and {df.shape[1]} columns.")
df

Dataframe now contains 31 rows and 3 columns.


,PostalCode,Borough,Neighborhood
1,M4A,Downtown Toronto(Regent Park / Harbourfront),North York(Lawrence Manor / Lawrence Heights)
4,M4B,"Downtown Toronto(Garden District, Ryerson)",North York(Glencairn)
7,M4C,Downtown Toronto(St. James Town),York(Humewood-Cedarvale)
10,M4E,Downtown Toronto(Berczy Park),York(Caledonia-Fairbanks)
13,M4G,Downtown Toronto(Central Bay Street),Downtown Toronto(Christie)
15,M1H,North York(Hillcrest Village),North York(Bathurst Manor / Wilson Heights / D...
16,M4H,Downtown Toronto(Richmond / Adelaide / King),West Toronto(Dufferin / Dovercourt Village)
18,M1J,North York(Fairview / Henry Farm / Oriole),North York(Northwood Park / York University)
19,M4J,Downtown Toronto(Harbourfront East / Union Sta...,West Toronto(Little Portugal / Trinity)
21,M1K,North York(Bayview Village),North York(Downsview)East (CFB Toronto)


group boroughs and neighborhoods by postal codes

In [16]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df

,PostalCode,Borough,Neighborhood
0,M1H,North York(Hillcrest Village),North York(Bathurst Manor / Wilson Heights / D...
1,M1J,North York(Fairview / Henry Farm / Oriole),North York(Northwood Park / York University)
2,M1K,North York(Bayview Village),North York(Downsview)East (CFB Toronto)
3,M1L,North York(York Mills / Silver Hills),North York(Downsview)West
4,M1M,North York(Willowdale / Newtonbrook),North York(Downsview)Central
5,M1N,North York(Willowdale)South,North York(Downsview)Northwest
6,M1P,North York(York Mills West),Not assigned
7,M1R,North York(Willowdale)West,Not assigned
8,M4A,Downtown Toronto(Regent Park / Harbourfront),North York(Lawrence Manor / Lawrence Heights)
9,M4B,"Downtown Toronto(Garden District, Ryerson)",North York(Glencairn)


rename neighborhoods that are 'Not assigned'

In [17]:
# 'Not assigned' neighborhoods are given the name of their corresponding borough
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned',
                              df['Borough'],
                              df['Neighborhood']
                             )

show dataframe

In [18]:
print(f"Cleansed dataframe contains {df.shape[0]} rows and {df.shape[1]} columns.")
df

Cleansed dataframe contains 31 rows and 3 columns.


,PostalCode,Borough,Neighborhood
0,M1H,North York(Hillcrest Village),North York(Bathurst Manor / Wilson Heights / D...
1,M1J,North York(Fairview / Henry Farm / Oriole),North York(Northwood Park / York University)
2,M1K,North York(Bayview Village),North York(Downsview)East (CFB Toronto)
3,M1L,North York(York Mills / Silver Hills),North York(Downsview)West
4,M1M,North York(Willowdale / Newtonbrook),North York(Downsview)Central
5,M1N,North York(Willowdale)South,North York(Downsview)Northwest
6,M1P,North York(York Mills West),North York(York Mills West)
7,M1R,North York(Willowdale)West,North York(Willowdale)West
8,M4A,Downtown Toronto(Regent Park / Harbourfront),North York(Lawrence Manor / Lawrence Heights)
9,M4B,"Downtown Toronto(Garden District, Ryerson)",North York(Glencairn)


In [19]:
df.to_csv('Toronto-Pt1.csv', index=False)

part 2

retrieve geographical coordinates using Geocoder

In [21]:
# a function that retrieves lat/long coordinates
def get_coords(postcode):
    return geocoder.arcgis(f'{postcode}, Toronto, Ontario').latlng

In [22]:
# get coordinates of all postal codes
postcodes = df['PostalCode'].tolist()
coords = [get_coords(code) for code in postcodes] # apply function
print("Coordinates are obtained.")

Coordinates are obtained.


In [23]:
# a new dataframe to store coordinates
coords_arcgis = df.copy()

# add columns for latitudes and longitudes
coords_arcgis['Latitude'] = [coord[0] for coord in coords]
coords_arcgis['Longitude'] = [coord[1] for coord in coords]
coords_arcgis.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1H,North York(Hillcrest Village),North York(Bathurst Manor / Wilson Heights / D...,43.76944,-79.23892
1,M1J,North York(Fairview / Henry Farm / Oriole),North York(Northwood Park / York University),43.74446,-79.23117
2,M1K,North York(Bayview Village),North York(Downsview)East (CFB Toronto),43.72582,-79.26461
3,M1L,North York(York Mills / Silver Hills),North York(Downsview)West,43.71289,-79.28506
4,M1M,North York(Willowdale / Newtonbrook),North York(Downsview)Central,43.72360,-79.23496


In [25]:
from google.colab import files
uploaded = files.upload()

Saving Geospatial_Coordinates.csv to Geospatial_Coordinates.csv


In [26]:
# load the coordinates file
coords_google = pd.read_csv('Geospatial_Coordinates.csv',
                            header=0,
                            names=['PostalCode', 'Latitude', 'Longitude']
                           )

# merge with the original dataframe
toronto = df.merge(coords_google, on='PostalCode', how='inner')
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1H,North York(Hillcrest Village),North York(Bathurst Manor / Wilson Heights / D...,43.773136,-79.239476
1,M1J,North York(Fairview / Henry Farm / Oriole),North York(Northwood Park / York University),43.744734,-79.239476
2,M1K,North York(Bayview Village),North York(Downsview)East (CFB Toronto),43.727929,-79.262029
3,M1L,North York(York Mills / Silver Hills),North York(Downsview)West,43.711112,-79.284577
4,M1M,North York(Willowdale / Newtonbrook),North York(Downsview)Central,43.716316,-79.239476


In [27]:
toronto.to_csv('Toronto-Pt2.csv', index=False)

part 3

visualizatio: Tronto neighborhoods

In [28]:
# get the coordinates of Toronto using Nominatim geocoder
geolocator = Nominatim(user_agent='tor_explorer')
location = geolocator.geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude
print(f"The geograpical coordinates of Toronto are {latitude}, {longitude}.")

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [29]:
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, nbhd in zip(toronto['Latitude'],
                                   toronto['Longitude'],
                                   toronto['Borough'],
                                   toronto['Neighborhood']):
    label = f"{nbhd}, {borough}"
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='purple',
                        fill=True,
                        fill_color='pink',
                        fill_opacity=0.7,
                        parse_html=False
                       ).add_to(map_tor)

map_tor

select broughs of interest

In [30]:
tor_boi = toronto[toronto['Borough'].str.contains('Toronto')]
tor_boi.reset_index(drop=True, inplace=True)
print(tor_boi['Borough'].unique().tolist())
tor_boi.head()

['Downtown Toronto(Regent Park / Harbourfront)', 'Downtown Toronto(Garden District, Ryerson)', 'Downtown Toronto(St. James Town)', 'Downtown Toronto(Berczy Park)', 'Downtown Toronto(Central Bay Street)', 'Downtown Toronto(Richmond / Adelaide / King)', 'Downtown Toronto(Harbourfront East / Union Station / Toronto Islands)', 'Downtown Toronto(Toronto Dominion Centre / Design Exchange)', 'Downtown Toronto(Commerce Court / Victoria Hotel)', 'Central Toronto(Roselawn)', 'Central Toronto(Forest Hill North & West)', 'Central Toronto(The Annex / North Midtown / Yorkville)', 'Downtown Toronto(University of Toronto / Harbord)', 'Downtown Toronto(Kensington Market / Chinatown / Grange Park)', 'Downtown Toronto(CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport)', 'Downtown TorontoStn A PO Boxes25 The Esplanade(Enclave of M5E)', 'Downtown Toronto(First Canadian Place / Underground city)']


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4A,Downtown Toronto(Regent Park / Harbourfront),North York(Lawrence Manor / Lawrence Heights),43.725882,-79.315572
1,M4B,"Downtown Toronto(Garden District, Ryerson)",North York(Glencairn),43.706397,-79.309937
2,M4C,Downtown Toronto(St. James Town),York(Humewood-Cedarvale),43.695344,-79.318389
3,M4E,Downtown Toronto(Berczy Park),York(Caledonia-Fairbanks),43.676357,-79.293031
4,M4G,Downtown Toronto(Central Bay Street),Downtown Toronto(Christie),43.709060,-79.363452


In [50]:
tor_boi

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4A,Downtown Toronto(Regent Park / Harbourfront),North York(Lawrence Manor / Lawrence Heights),43.725882,-79.315572
1,M4B,"Downtown Toronto(Garden District, Ryerson)",North York(Glencairn),43.706397,-79.309937
2,M4C,Downtown Toronto(St. James Town),York(Humewood-Cedarvale),43.695344,-79.318389
3,M4E,Downtown Toronto(Berczy Park),York(Caledonia-Fairbanks),43.676357,-79.293031
4,M4G,Downtown Toronto(Central Bay Street),Downtown Toronto(Christie),43.709060,-79.363452
5,M4H,Downtown Toronto(Richmond / Adelaide / King),West Toronto(Dufferin / Dovercourt Village),43.705369,-79.349372
6,M4J,Downtown Toronto(Harbourfront East / Union Sta...,West Toronto(Little Portugal / Trinity),43.685347,-79.338106
7,M4K,Downtown Toronto(Toronto Dominion Centre / Des...,West Toronto(Brockton / Parkdale Village / Exh...,43.679557,-79.352188
8,M4L,Downtown Toronto(Commerce Court / Victoria Hotel),North York(North Park / Maple Leaf Park / Upwo...,43.668999,-79.315572
9,M4N,Central Toronto(Roselawn),York(Runnymede / The Junction North),43.728020,-79.388790


In [31]:
map_tor_boi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, nbhd in zip(tor_boi['Latitude'],
                                   tor_boi['Longitude'],
                                   tor_boi['Borough'],
                                   tor_boi['Neighborhood']):
    label = folium.Popup(f"{nbhd}, {borough}", parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='purple',
                        fill=True,
                        fill_color='pink',
                        fill_opacity=0.7,
                        parse_html=False
                       ).add_to(map_tor_boi)
map_tor_boi

explore a particular neighborhood

In [51]:
# select a neighborhood and its corresponding coordinates
noi = 'West Toronto(Runnymede / Swansea)'
noi_idx = int(12)
noi_lat =float(43.704324)
noi_lng = float(-79.388790)
print(f"Latitude and longitude values of {noi} are {noi_lat}, {noi_lng}.")



Latitude and longitude values of West Toronto(Runnymede / Swansea) are 43.704324, -79.38879.


In [52]:
# define Foursquare credentials and parameters
client_id = 'not show'
client_secret = 'not show'
version = '20210101'
limit = 100
radius = 500

In [53]:
# specify url
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    client_id, 
    client_secret, 
    version, 
    noi_lat, 
    noi_lng, 
    radius, 
    limit)

# make an HTTP request and
# store the response in a variable 'results'
results = requests.get(url).json()
print("Request successful.")

Request successful.


In [54]:
# a function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [55]:
# get all the venues, flatten JSON into a dataframe
venues_json = results['response']['groups'][0]['items'] 
venues = pd.json_normalize(venues_json)

# filter relevant columns
cols_filtered = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues = venues.loc[:, cols_filtered]

# apply categorization of venues
venues['venue.categories'] = venues.apply(get_category_type, axis=1)

# rename columns
venues.columns = ['Name', 'Category', 'Latitude', 'Longitude']

print(f"{venues.shape[0]} venues were returned by Foursquare.")
venues.head(10)

28 venues were returned by Foursquare.


,Name,Category,Latitude,Longitude
0,Jules Cafe Patisserie,Dessert Shop,43.704138,-79.388413
1,Marigold Indian Bistro,Indian Restaurant,43.702881,-79.388008
2,Zee Grill,Seafood Restaurant,43.704985,-79.388476
3,Thobors Boulangerie Patisserie Café,Café,43.704514,-79.388616
4,Sakae Sushi,Sushi Restaurant,43.704944,-79.388704
5,Thai Spicy House,Thai Restaurant,43.701962,-79.387513
6,XO Gelato,Dessert Shop,43.705177,-79.388793
7,Viva Napoli,Pizza Place,43.705752,-79.389125
8,June Rowlands Park,Park,43.700517,-79.389189
9,Mastermind Toys,Toy / Game Store,43.704839,-79.388546


In [56]:
noi_top_venues = venues['Category'].value_counts()
print(f"The top venues in {noi} are:\n")
noi_top_venues.loc[noi_top_venues > 1]

The top venues in West Toronto(Runnymede / Swansea) are:



Pizza Place         3
Sushi Restaurant    2
Gym                 2
Coffee Shop         2
Dessert Shop        2
Toy / Game Store    2
Sandwich Place      2
Name: Category, dtype: int64

exlpore venues in all neighborhoods

In [57]:
def get_venues(names, lats, lngs, radius=500, limit=100):
    venues_list = []
    for name, lat, lng in zip(names, lats, lngs):
        # specify the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id,
            client_secret,
            version,
            lat,
            lng,
            radius,
            limit)
        # make the request, store the response
        results = requests.get(url).json()['response']['groups'][0]['items']
        # extract relevant information from each venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    # populate the dataframe with venues list
    venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    venues.columns = ['Neighborhood',
                      'Nbhd Latitude',
                      'Nbhd Longitude',
                      'Venue',
                      'Venue Latitude',
                      'Venue Longitude',
                      'Venue Category']
    return(venues)

In [58]:
# run the above function on each neighborhood,
# store the result as a new dataframe
tor_venues = get_venues(
    tor_boi['Neighborhood'],
    tor_boi['Latitude'],
    tor_boi['Longitude']
)
tor_venues.head()

,Neighborhood,Nbhd Latitude,Nbhd Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York(Lawrence Manor / Lawrence Heights),43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
1,North York(Lawrence Manor / Lawrence Heights),43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
2,North York(Lawrence Manor / Lawrence Heights),43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
3,North York(Lawrence Manor / Lawrence Heights),43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
4,North York(Glencairn),43.706397,-79.309937,Jawny Bakers,43.705783,-79.312913,Gastropub


In [59]:
print(f"In all {tor_venues['Neighborhood'].nunique()} Toronto neighborhoods,",
      f"there's a total of {tor_venues.shape[0]} venues",
      f"across {tor_venues['Venue Category'].nunique()} different categories."
     )
tor_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count().reset_index()

In all 17 Toronto neighborhoods, there's a total of 247 venues across 87 different categories.


,Neighborhood,Venue
0,Downtown Toronto(CN Tower / King and Spadina /...,11
1,Downtown Toronto(Christie),26
2,Downtown Toronto(First Canadian Place / Underg...,36
3,Downtown Toronto(Kensington Market / Chinatown...,2
4,Downtown TorontoStn A PO Boxes25 The Esplanade...,4
5,North York(Glencairn),12
6,North York(Lawrence Manor / Lawrence Heights),4
7,North York(North Park / Maple Leaf Park / Upwo...,19
8,West Toronto(Brockton / Parkdale Village / Exh...,42
9,West Toronto(Dufferin / Dovercourt Village),19


analyze each neighborhood

In [60]:
# one-hot encoding for each venue category
tor_onehot = pd.get_dummies(tor_venues['Venue Category'], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 
tor_onehot = tor_onehot.groupby('Neighborhood').mean().reset_index()

print(f"Dataframe size: {tor_onehot.shape[0]} rows, {tor_onehot.shape[1]} columns")
tor_onehot.head()

Dataframe size: 17 rows, 87 columns


,Neighborhood,American Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Beer Store,Board Shop,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Frozen Yogurt Shop,Gas Station,Gastropub,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,...,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Liquor Store,Lounge,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Music Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Portuguese Restaurant,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Skating Rink,Snack Place,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Video Store,Vietnamese Restaurant,Warehouse Store,Yoga Studio
0,Downtown Toronto(CN Tower / King and Spadina /...,0.090909,0.0,0.090909,0.000000,0.090909,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.181818,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.090909,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.090909,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.0,0.000000,0.0,0.000000,0.090909,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.090909,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.090909,0.0,0.000000
1,Downtown Toronto(Christie),0.000000,0.0,0.000000,0.000000,0.076923,0.038462,0.0,0.076923,0.038462,0.0,0.076923,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.115385,0.0,0.0,0.0,0.000000,0.0,0.038462,0.000000,0.0,0.0,0.038462,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.038462,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.038462,0.0,0.0,0.038462,0.0,0.0,0.038462,0.000000,0.038462,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.038462,0.0,0.0,0.000000,0.0,0.115385,0.038462,0.0,0.038462,0.076923,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000
2,Downtown Toronto(First Canadian Place / Underg...,0.000000,0.0,0.000000,0.055556,0.027778,0.027778,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.055556,0.027778,0.027778,0.0,0.111111,0.0,0.0,0.0,0.027778,0.0,0.000000,0.027778,0.0,0.0,0.000000,0.0,0.027778,0.0,0.000000,0.0,0.0,0.027778,0.0,0.027778,0.0,0.0,...,0.027778,0.027778,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.027778,0.027778,0.027778,0.055556,0.027778,0.0,0.055556,0.0,0.055556,0.027778,0.0,0.0,0.027778,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.027778,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.027778
3,Downtown Toronto(Kensington Market / Chinatown...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.500000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.5,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000
4,Downtown TorontoStn A PO Boxes25 The Esplanade...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.500000,0.000000,0.000000,0.000000,0.250000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.000000


In [61]:
# a function to sort venues in a descending order of frequency
def top_venues(row, num_venues):
    row_cats = row.iloc[1:]
    row_cats_sorted = row_cats.sort_values(ascending=False)
    return row_cats_sorted.index.values[0:num_venues]

In [62]:
num_venues = 10 # number of top venues
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
cols = ['Neighborhood']
for i in np.arange(num_venues):
    try:
        cols.append(f"{i+1}{indicators[i]} Most Common Venue")
    except:
        cols.append(f"{i+1}th Most Common Venue")

# create a dataframe of 10 most common venues by neighborhood
tor_common = pd.DataFrame(columns=cols)
tor_common['Neighborhood'] = tor_onehot['Neighborhood']

for i in np.arange(tor_onehot.shape[0]):
    tor_common.iloc[i, 1:] = top_venues(tor_onehot.iloc[i, :], num_venues)

tor_common.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto(CN Tower / King and Spadina /...,Coffee Shop,American Restaurant,Bank,Sandwich Place,Fried Chicken Joint,Sushi Restaurant,Light Rail Station,Pizza Place,Bagel Shop,Vietnamese Restaurant
1,Downtown Toronto(Christie),Coffee Shop,Sporting Goods Shop,Bank,Breakfast Spot,Sushi Restaurant,Burger Joint,Fish & Chips Shop,Liquor Store,Mexican Restaurant,Dessert Shop
2,Downtown Toronto(First Canadian Place / Underg...,Coffee Shop,Pizza Place,Italian Restaurant,Pub,Café,Restaurant,Bakery,Japanese Restaurant,Jewelry Store,Diner
3,Downtown Toronto(Kensington Market / Chinatown...,Restaurant,Tennis Court,Yoga Studio,Food & Drink Shop,Department Store,Dessert Shop,Diner,Discount Store,Fast Food Restaurant,Fish & Chips Shop
4,Downtown TorontoStn A PO Boxes25 The Esplanade...,Park,Playground,Trail,Flower Shop,Department Store,Dessert Shop,Diner,Discount Store,Fast Food Restaurant,Fish & Chips Shop


cluster neighborhoods using K-means algorithm

In [63]:
Ks = 5 # number of clusters
X = tor_onehot.drop('Neighborhood', 1) # select features

model = KMeans(n_clusters=Ks, random_state=0).fit(X)
model.labels_ # cluster labels generated for each row

array([0, 0, 0, 3, 1, 0, 4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2], dtype=int32)

In [64]:
tor_common.insert(0, 'Cluster Label', model.labels_)
tor_merged = tor_boi.copy()
tor_merged = tor_merged.join(tor_common.set_index('Neighborhood'), on='Neighborhood')
tor_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4A,Downtown Toronto(Regent Park / Harbourfront),North York(Lawrence Manor / Lawrence Heights),43.725882,-79.315572,4,Hockey Arena,Portuguese Restaurant,Pizza Place,Coffee Shop,Dessert Shop,Diner,Discount Store,Fast Food Restaurant,Fish & Chips Shop,Flea Market
1,M4B,"Downtown Toronto(Garden District, Ryerson)",North York(Glencairn),43.706397,-79.309937,0,Pizza Place,Pharmacy,Athletics & Sports,Gym / Fitness Center,Pet Store,Bank,Intersection,Breakfast Spot,Gastropub,Café
2,M4C,Downtown Toronto(St. James Town),York(Humewood-Cedarvale),43.695344,-79.318389,0,Curling Ice,Intersection,Skating Rink,Beer Store,Dance Studio,Video Store,Athletics & Sports,Gym / Fitness Center,Flower Shop,Dessert Shop
3,M4E,Downtown Toronto(Berczy Park),York(Caledonia-Fairbanks),43.676357,-79.293031,0,Health Food Store,Coffee Shop,Pub,Yoga Studio,Flower Shop,Department Store,Dessert Shop,Diner,Discount Store,Fast Food Restaurant
4,M4G,Downtown Toronto(Central Bay Street),Downtown Toronto(Christie),43.709060,-79.363452,0,Coffee Shop,Sporting Goods Shop,Bank,Breakfast Spot,Sushi Restaurant,Burger Joint,Fish & Chips Shop,Liquor Store,Mexican Restaurant,Dessert Shop


In [65]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(Ks)
ys = [i + x + (i*x)**2 for i in range(Ks)]
colors_array = cm.gist_rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lng, nbhd, cluster in zip(tor_merged['Latitude'],
                                   tor_merged['Longitude'],
                                   tor_merged['Neighborhood'],
                                   tor_merged['Cluster Label']):
    label = folium.Popup(f"Cluster {cluster}: {nbhd}", parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color=rainbow[cluster-1],
                        fill=True,
                        fill_color=rainbow[cluster-1],
                        fill_opacity=0.5
                       ).add_to(map_clusters)
map_clusters

examine clusters

In [66]:
cluster0 = tor_merged.loc[tor_merged['Cluster Label'] == 0,
                          tor_merged.columns[[2] + list(range(6, tor_merged.shape[1]))]
                         ].reset_index(drop=True)
cluster0

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York(Glencairn),Pizza Place,Pharmacy,Athletics & Sports,Gym / Fitness Center,Pet Store,Bank,Intersection,Breakfast Spot,Gastropub,Café
1,York(Humewood-Cedarvale),Curling Ice,Intersection,Skating Rink,Beer Store,Dance Studio,Video Store,Athletics & Sports,Gym / Fitness Center,Flower Shop,Dessert Shop
2,York(Caledonia-Fairbanks),Health Food Store,Coffee Shop,Pub,Yoga Studio,Flower Shop,Department Store,Dessert Shop,Diner,Discount Store,Fast Food Restaurant
3,Downtown Toronto(Christie),Coffee Shop,Sporting Goods Shop,Bank,Breakfast Spot,Sushi Restaurant,Burger Joint,Fish & Chips Shop,Liquor Store,Mexican Restaurant,Dessert Shop
4,West Toronto(Dufferin / Dovercourt Village),Sandwich Place,Indian Restaurant,Grocery Store,Burger Joint,Pizza Place,Middle Eastern Restaurant,Discount Store,Coffee Shop,Fast Food Restaurant,Liquor Store
5,West Toronto(Brockton / Parkdale Village / Exh...,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Bubble Tea Shop,Café,Caribbean Restaurant,Brewery,Dessert Shop
6,North York(North Park / Maple Leaf Park / Upwo...,Park,Fast Food Restaurant,Steakhouse,Pet Store,Pizza Place,Pub,Restaurant,Sandwich Place,Liquor Store,Light Rail Station
7,West Toronto(High Park / The Junction South),Hotel,Breakfast Spot,Department Store,Sandwich Place,Park,Playground,Food & Drink Shop,Gym,Gym / Fitness Center,Pizza Place
8,West Toronto(Parkdale / Roncesvalles),Clothing Store,Coffee Shop,Sporting Goods Shop,Fast Food Restaurant,Diner,Park,Rental Car Location,Chinese Restaurant,Café,Spa
9,West Toronto(Runnymede / Swansea),Pizza Place,Sushi Restaurant,Sandwich Place,Toy / Game Store,Coffee Shop,Dessert Shop,Gym,Brewery,Seafood Restaurant,Café


In [67]:
# get all the venue categories in the cluster
cl0_venues_lists = cluster0.iloc[:, 1:].values.tolist()

# flatten the resulting nested list
cl0_venues = [venue for sublist in cl0_venues_lists for venue in sublist]

# count unique values
pd.Series(cl0_venues).value_counts()

Coffee Shop           8
Pizza Place           7
Café                  5
Dessert Shop          5
Sandwich Place        5
                     ..
Yoga Studio           1
Seafood Restaurant    1
Toy / Game Store      1
Health Food Store     1
Bagel Shop            1
Length: 62, dtype: int64

In [68]:
cluster1 = tor_merged.loc[tor_merged['Cluster Label'] == 1,
                          tor_merged.columns[[2] + list(range(6, tor_merged.shape[1]))]
                         ].reset_index(drop=True)
cluster1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,West Toronto(Little Portugal / Trinity),Metro Station,Convenience Store,Coffee Shop,Park,Food & Drink Shop,Diner,Discount Store,Fast Food Restaurant,Fish & Chips Shop,Flea Market
1,Downtown TorontoStn A PO Boxes25 The Esplanade...,Park,Playground,Trail,Flower Shop,Department Store,Dessert Shop,Diner,Discount Store,Fast Food Restaurant,Fish & Chips Shop


In [69]:
# get all the venue categories in the cluster
cl1_venues_lists = cluster1.iloc[:, 1:].values.tolist()

# flatten the resulting nested list
cl1_venues = [venue for sublist in cl1_venues_lists for venue in sublist]

# count unique values
pd.Series(cl1_venues).value_counts()

Fish & Chips Shop       2
Park                    2
Discount Store          2
Fast Food Restaurant    2
Diner                   2
Food & Drink Shop       1
Dessert Shop            1
Metro Station           1
Convenience Store       1
Flower Shop             1
Coffee Shop             1
Flea Market             1
Trail                   1
Playground              1
Department Store        1
dtype: int64

In [70]:
cluster2 = tor_merged.loc[tor_merged['Cluster Label'] == 2,
                          tor_merged.columns[[2] + list(range(6, tor_merged.shape[1]))]
                         ].reset_index(drop=True)

cl2_venues_lists = cluster2.iloc[:, 1:].values.tolist()
cl2_venues = [venue for sublist in cl2_venues_lists for venue in sublist]
pd.Series(cl2_venues).value_counts()

Diner                   1
Discount Store          1
Bus Line                1
Park                    1
Swim School             1
Yoga Studio             1
Dessert Shop            1
Gym / Fitness Center    1
Flower Shop             1
Fast Food Restaurant    1
dtype: int64

In [71]:
cluster3 = tor_merged.loc[tor_merged['Cluster Label'] == 3,
                          tor_merged.columns[[2] + list(range(6, tor_merged.shape[1]))]
                         ].reset_index(drop=True)

cl3_venues_lists = cluster3.iloc[:, 1:].values.tolist()
cl3_venues = [venue for sublist in cl3_venues_lists for venue in sublist]
pd.Series(cl3_venues).value_counts()

Diner                   1
Discount Store          1
Restaurant              1
Fast Food Restaurant    1
Tennis Court            1
Food & Drink Shop       1
Dessert Shop            1
Yoga Studio             1
Fish & Chips Shop       1
Department Store        1
dtype: int64

In [72]:
cluster4 = tor_merged.loc[tor_merged['Cluster Label'] == 4,
                          tor_merged.columns[[2] + list(range(6, tor_merged.shape[1]))]
                         ].reset_index(drop=True)

cl4_venues_lists = cluster4.iloc[:, 1:].values.tolist()
cl4_venues = [venue for sublist in cl4_venues_lists for venue in sublist]
pd.Series(cl4_venues).value_counts()

Diner                    1
Discount Store           1
Pizza Place              1
Flea Market              1
Fast Food Restaurant     1
Portuguese Restaurant    1
Coffee Shop              1
Dessert Shop             1
Fish & Chips Shop        1
Hockey Arena             1
dtype: int64